In [4]:
import cv2
import dlib
import numpy as np

# Load the pre-trained model for face detection from dlib
detector = dlib.get_frontal_face_detector()

# Load the pre-trained MobileNetV2-based SSD model for face detection from OpenCV
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt",
    "res10_300x300_ssd_iter_140000.caffemodel"
)

# Open a connection to the webcam (0 represents the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()

    # Resize the frame to a fixed width for better performance
    width = 800
    height = int(frame.shape[0] * (width / frame.shape[1]))
    resized_frame = cv2.resize(frame, (width, height))

    # Convert the frame to grayscale (for dlib)
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using dlib
    faces_dlib = detector(gray)

    # Detect faces using the SSD model
    blob = cv2.dnn.blobFromImage(resized_frame, 1.0, (300, 300), (104, 177, 123))
    net.setInput(blob)
    detections = net.forward()

    # Loop over the face detections and draw rectangles around the faces
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > 0.5:  # You can adjust the confidence threshold as needed
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (startX, startY, endX, endY) = box.astype(int)

            # Draw the face rectangle using OpenCV
            cv2.rectangle(resized_frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Display the resulting frame with detected faces
    cv2.imshow("Face Detection", resized_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
